In [2]:
pip install tensorflow keras mediapipe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 55.5 MB/s eta 0:00:00


In [7]:
pip install tensorflow-addons

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 12.4 MB/s eta 0:00:00
  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.4.1
    Uninstalling typeguard-4.4.1:
      Successfully uninstalled typeguard-4.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
inflect 7.5.0 requires typeguard>=4.0.1, but you have typeguard 2.13.3 which is incompatible.


In [13]:
import os
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input, Dense, Flatten, Concatenate, LayerNormalization, MultiHeadAttention, Dropout
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm
from tensorflow.keras import mixed_precision


# Step 1: GPU Utilization and Mixed Precision
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    for device in physical_devices:
        try:
            tf.config.experimental.set_memory_growth(device, True)
            tf.config.set_logical_device_configuration(
                device, [tf.config.LogicalDeviceConfiguration(memory_limit=10000)]
            )
            print("Configured GPU with a memory limit of 10,000 MB.")
        except Exception as e:
            print(f"Error configuring GPU: {e}")
else:
    print("No GPU detected, running on CPU.")

try:
    # Enable mixed precision globally
    policy = mixed_precision.Policy('mixed_float16')
    mixed_precision.set_global_policy(policy)
    print(f"Mixed precision policy set to: {policy}")

except ValueError:
    print("Mixed precision not supported, running with default precision.")

Error configuring GPU: Cannot set memory growth on device when virtual devices configured
Mixed precision policy set to: <DTypePolicy "mixed_float16">


In [17]:
# Step 2: Load and Prepare Features with Parallel Processing
features_folder = 'drive/MyDrive/SP_cup/features/'
feature_files = {
    'spatial': ['merged_spatial_fake.pkl', 'spatial_features_real_images.pkl'],
    'landmarks': ['merged_facial_fake.pkl', 'merged_landmarks_real.pkl']
}

def load_features(file_list, folder):
    """Load features from a list of .pkl files."""
    data = []
    for file in file_list:
        with open(os.path.join(folder, file), 'rb') as f:
            data.append(pickle.load(f))
    return np.vstack(data)  # Stack features vertically

# Parallel feature loading
def load_all_features():
    with ProcessPoolExecutor() as executor:
        spatial_future = executor.submit(load_features, feature_files['spatial'], features_folder)
        landmark_future = executor.submit(load_features, feature_files['landmarks'], features_folder)
        spatial = spatial_future.result()
        landmarks = landmark_future.result()
    return spatial, landmarks

spatial_features, landmark_features = load_all_features()

# Create labels
labels = np.array([1] * (len(spatial_features) // 2) + [0] * (len(spatial_features) // 2))  # Adjust for dataset sizes
y = to_categorical(labels, num_classes=2)


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (16,) + inhomogeneous part.

In [10]:
# Step 3: Define Transformer Block
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential([
            Dense(ff_dim, activation='relu'),
            Dense(embed_dim),
        ])
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)


In [11]:
# Step 4: Define the Model
def build_model(spatial_shape, landmark_shape):
    spatial_input = Input(shape=(spatial_shape,))
    spatial_flat = Flatten()(spatial_input)

    landmark_input = Input(shape=(landmark_shape,))
    landmark_embed = Dense(128, activation='relu')(landmark_input)
    landmark_transformer = TransformerBlock(embed_dim=128, num_heads=4, ff_dim=512)(landmark_embed)

    combined = Concatenate()([spatial_flat, landmark_transformer])
    combined_dense = Dense(256, activation='relu')(combined)
    output = Dense(2, activation='softmax', dtype='float32')(combined_dense)  # Float32 for mixed precision compatibility

    model = Model(inputs=[spatial_input, landmark_input], outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [15]:
from sklearn.model_selection import KFold


In [16]:
# Step 5: K-Fold Cross-Validation with k=2
kf = KFold(n_splits=2, shuffle=True, random_state=42)
fold = 1
spatial_shape = spatial_features.shape[1]
landmark_shape = landmark_features.shape[1]

# Initialize variables to store fold metrics
fold_accuracies = []
fold_losses = []

for train_index, val_index in kf.split(spatial_features):
    print(f"\n--- Fold {fold} ---")

    spatial_train, spatial_val = spatial_features[train_index], spatial_features[val_index]
    landmark_train, landmark_val = landmark_features[train_index], landmark_features[val_index]
    y_train, y_val = y[train_index], y[val_index]

    model = build_model(spatial_shape, landmark_shape)

    # Callbacks for progress saving
    checkpoint_cb = ModelCheckpoint(f'model_checkpoint_fold{fold}.h5', save_best_only=True)
    logger_cb = CSVLogger(f'training_log_fold{fold}.csv', append=True)

    # Train model
    with tqdm(total=50, desc=f"Training Fold {fold}") as pbar:
        history = model.fit(
            [spatial_train, landmark_train], y_train,
            validation_data=([spatial_val, landmark_val], y_val),
            batch_size=32,
            epochs=50,
            shuffle=True,
            callbacks=[checkpoint_cb, logger_cb],
            verbose=0
        )
        pbar.update(50)

    # Evaluate model
    val_loss, val_acc = model.evaluate([spatial_val, landmark_val], y_val, verbose=0)
    print(f"Validation Accuracy for Fold {fold}: {val_acc * 100:.2f}%")
    fold_accuracies.append(val_acc * 100)
    fold_losses.append(val_loss)

    fold += 1

NameError: name 'spatial_features' is not defined

In [ ]:
# Step 6: Aggregate and Display Metrics
print("\n--- K-Fold Cross-Validation Results ---")
print(f"Average Accuracy: {np.mean(fold_accuracies):.2f}%")
print(f"Average Loss: {np.mean(fold_losses):.4f}")
print(f"Accuracies per Fold: {fold_accuracies}")